# Cross-Country Solar Comparison (Benin, Sierra Leone, Togo)

This notebook synthesizes cleaned datasets for Benin, Sierra Leone, and Togo to compare solar resource characteristics and operational patterns.

Objectives:
- Load each country’s cleaned CSV from `../data/`
- Plot side-by-side boxplots for GHI, DNI, DHI
- Build a summary table (mean, median, std) per country and metric
- Run one-way ANOVA (and Kruskal–Wallis) on GHI to test statistical differences
- Capture 3 key observations in markdown-style output
- Visual summary: bar chart ranking countries by average GHI

KPI checklist:
- All three countries included (when files are available)
- Correct p-values reported
- Actionable insights
- Summary table for mean/median/std

In [ ]:
# Imports and setup
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import stats
from IPython.display import display, Markdown
import warnings

sns.set(style="whitegrid", context="notebook")
plt.rcParams["figure.figsize"] = (10, 5)
warnings.filterwarnings("ignore")

DATA_DIR = Path("../data")
COUNTRY_FILES = {
    "Benin": "benin_clean.csv",
    "Sierra Leone": "sierra_leone_clean.csv",
    "Togo": "togo_clean.csv",
}

METRICS = ["GHI", "DNI", "DHI"]

In [ ]:
# Load cleaned datasets (if available)

def load_country_df(country_name: str, filename: str) -> pd.DataFrame:
    fp = DATA_DIR / filename
    if not fp.exists():
        print(f"[WARN] Missing file for {country_name}: {fp}")
        return pd.DataFrame()
    df = pd.read_csv(fp)
    # Parse timestamp if present
    if 'Timestamp' in df.columns:
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
    df['Country'] = country_name
    return df

frames = []
for country, file in COUNTRY_FILES.items():
    frames.append(load_country_df(country, file))

df_all = pd.concat([f for f in frames if not f.empty], ignore_index=True)

available_countries = sorted(df_all['Country'].unique()) if not df_all.empty else []
print(f"Available countries: {available_countries}")

# Keep only required metrics if present
metrics_present = [m for m in METRICS if m in df_all.columns]
if len(metrics_present) < 1:
    print("[ERROR] None of the required metrics (GHI, DNI, DHI) are present.")
else:
    print(f"Metrics present: {metrics_present}")

In [ ]:
# Boxplots: GHI, DNI, DHI by Country
if df_all.empty or len(available_countries) < 1:
    print("No data available to plot.")
else:
    n = len(metrics_present)
    fig, axes = plt.subplots(1, n, figsize=(6*n, 5))
    if n == 1:
        axes = [axes]
    for i, metric in enumerate(metrics_present):
        ax = axes[i]
        sns.boxplot(data=df_all, x="Country", y=metric, palette="Set2", ax=ax)
        ax.set_title(f"{metric} by Country")
        ax.set_xlabel("")
        ax.set_ylabel(f"{metric} (units)")
        ax.tick_params(axis='x', rotation=20)
    plt.tight_layout()
    plt.show()

In [ ]:
# Summary table: mean, median, std for GHI, DNI, DHI by Country
if df_all.empty or len(available_countries) < 1:
    print("No data available for summary table.")
else:
    summary_table = (
        df_all.groupby("Country")[metrics_present]
              .agg(["mean", "median", "std"])
              .round(2)
    )
    display(summary_table)


In [ ]:
# Statistical testing on GHI (ANOVA and Kruskal–Wallis)
if df_all.empty or 'GHI' not in df_all.columns or len(available_countries) < 2:
    print("Not enough data/countries to run statistical tests on GHI.")
else:
    # Build groups by country
    groups = [df_all.loc[df_all['Country'] == c, 'GHI'].dropna().values for c in available_countries]
    sizes = {c: len(g) for c, g in zip(available_countries, groups)}
    print(f"Sample sizes: {sizes}")

    # Only run tests if each group has at least 2 samples
    if any(len(g) < 2 for g in groups):
        print("Insufficient samples in one or more groups for statistical tests.")
    else:
        # One-way ANOVA (parametric)
        try:
            f_stat, p_anova = stats.f_oneway(*groups)
            print(f"ANOVA: F = {f_stat:.3f}, p = {p_anova:.4g}")
        except Exception as e:
            p_anova = np.nan
            print(f"ANOVA failed: {e}")

        # Kruskal–Wallis (non-parametric)
        try:
            h_stat, p_kw = stats.kruskal(*groups)
            print(f"Kruskal–Wallis: H = {h_stat:.3f}, p = {p_kw:.4g}")
        except Exception as e:
            p_kw = np.nan
            print(f"Kruskal–Wallis failed: {e}")

        # Interpretation
        def sig_label(p):
            if pd.isna(p):
                return "n/a"
            return "significant" if p < 0.05 else "not significant"

        print("Interpretation:")
        print(f"- ANOVA difference is {sig_label(p_anova)} at alpha=0.05")
        print(f"- Kruskal–Wallis difference is {sig_label(p_kw)} at alpha=0.05")

In [ ]:
# Key observations (3 bullets)
if df_all.empty or len(available_countries) < 1 or len(metrics_present) < 1:
    print("No data available to summarize.")
else:
    bullets = []
    # GHI-based observations
    if 'GHI' in df_all.columns:
        stats_ghi = df_all.groupby('Country')['GHI'].agg(['mean','median','std'])
        top_median_country = stats_ghi['median'].idxmax()
        top_median_val = stats_ghi.loc[top_median_country, 'median']
        top_var_country = stats_ghi['std'].idxmax()
        top_var_val = stats_ghi.loc[top_var_country, 'std']
        bullets.append(f"{top_median_country} shows the highest median GHI (~{top_median_val:.1f} W/m²).")
        bullets.append(f"{top_var_country} exhibits the greatest GHI variability (std ~{top_var_val:.1f} W/m²).")
    # DNI-based observation if available
    if 'DNI' in df_all.columns:
        stats_dni = df_all.groupby('Country')['DNI'].agg(['mean','median'])
        top_mean_dni_country = stats_dni['mean'].idxmax()
        top_mean_dni_val = stats_dni.loc[top_mean_dni_country, 'mean']
        bullets.append(f"{top_mean_dni_country} ranks highest on average DNI (~{top_mean_dni_val:.1f} W/m²).")
    elif 'DHI' in df_all.columns:
        stats_dhi = df_all.groupby('Country')['DHI'].agg(['mean','median'])
        top_mean_dhi_country = stats_dhi['mean'].idxmax()
        top_mean_dhi_val = stats_dhi.loc[top_mean_dhi_country, 'mean']
        bullets.append(f"{top_mean_dhi_country} ranks highest on average DHI (~{top_mean_dhi_val:.1f} W/m²).")

    text = "\n".join([f"- {b}" for b in bullets[:3]])
    display(Markdown("## Key Observations\n" + text))

In [ ]:
# (Bonus) Bar chart: rank countries by average GHI
if df_all.empty or 'GHI' not in df_all.columns or len(available_countries) < 1:
    print("No data available to rank by average GHI.")
else:
    avg_ghi = (
        df_all.groupby('Country')['GHI']
              .mean()
              .sort_values(ascending=False)
    )
    ax = sns.barplot(x=avg_ghi.index, y=avg_ghi.values, palette="viridis")
    ax.set_title("Average GHI by Country (Higher is Better)")
    ax.set_xlabel("")
    ax.set_ylabel("Average GHI (W/m²)")
    plt.xticks(rotation=15)
    plt.tight_layout()
    plt.show()